TRADE IN GOODS STATISTICS:Employee size_value

In [1]:
from gssutils import *
if is_interactive():
    scraper = Scraper('https://www.uktradeinfo.com/Statistics/OverseasTradeStatistics/AboutOverseastradeStatistics/Pages/OTSReports.aspx')
    display(scraper)
    scraper.select_dataset(title=lambda x: x.startswith('UK Trade in Goods by Business Characteristics'))
    display(scraper)
    idbrs = sorted(
        [dist for dist in scraper.distributions if dist.title.startswith('IDBR OTS tables')],
        key=lambda d: d.title, reverse=True)
    idbr = idbrs[0]
    display(idbr.title)
    tabs = {tab.name: tab for tab in idbr.as_databaker()}
    display(tabs.keys())

## HM Revenue & Customs uktradeinfo - OTS Reports

This is a catalog of datasets; choose one from the following:

* Midlands Regional Trade Statistics data 2014-2016
* 2015 UK VAT Registered Importer and Exporter Population
* Overseas Trade Statistics broken down by English Growth Hub areas - EU and non-EU Exports 2015
* UK Trade in Goods by Business Characteristics - Experimental Statistics
* 2010 Quinquennial Review of the UK Ancillary Cost Survey (ACS) full report
* Quality Report
* UK Statistics article
* 2009 Intrastat Triennial Final Report
* EDICOM Technical Implementation Report
* 2005 Intrastat Triennial Review V2.0
* EDICOM Report
* EDICOM Report

## UK Trade in Goods by Business Characteristics - Experimental Statistics

### Distributions

1. UK Trade in Goods by Business Characteristics - Experimental Statistics ([application/pdf](https://www.uktradeinfo.com/Statistics/Documents/Reports/IDBR_OTS_2015.pdf))
1. IDBR OTS tables 2015 ([MS Excel Spreadsheet](https://www.uktradeinfo.com/Statistics/Documents/Reports/IDBR_OTS_tables_2015.xls))
1. IDBR OTS Country data (expert users) 2015 ([MS Excel Spreadsheet](https://www.uktradeinfo.com/Statistics/Documents/Reports/IDBR_OTS_%20Country_data_expert_2015.xls))
1. UK Trade in Goods by Business Characteristics - Experimental Statistics ([application/pdf](https://www.uktradeinfo.com/Statistics/Documents/Reports/IDBR_OTS_2014.pdf))
1. IDBR OTS tables 2014 ([MS Excel Spreadsheet](https://www.uktradeinfo.com/Statistics/Documents/Reports/IDBR_OTS_tables_2014.xls))
1. IDBR OTS Country data (expert users) 2014 ([MS Excel Spreadsheet](https://www.uktradeinfo.com/Statistics/Documents/Reports/IDBR_OTS_%20Country_data_expert_2014.xls))
1. UK Trade in Goods by Business Characteristics - Experimental Statistics ([application/pdf](https://www.uktradeinfo.com/Statistics/Documents/Reports/IDBR_OTS_2013.pdf))
1. IDBR OTS tables 2013 ([MS Excel Spreadsheet](https://www.uktradeinfo.com/Statistics/Documents/Reports/IDBR_OTS_tables_2013.xls))
1. IDBR OTS Country data (expert users) 2013 ([MS Excel Spreadsheet](https://www.uktradeinfo.com/Statistics/Documents/Reports/IDBR_OTS_Country_data%20_expert_2013.xls))
1. UK Trade in Goods by Business Characteristics - Experimental Statistics ([application/pdf](https://www.uktradeinfo.com/Statistics/OverseasTradeStatistics/AboutOverseastradeStatistics/Documents/IDBR_OTS_2012.pdf))
1. IDBR OTS tables 2009 - 2012 ([MS Excel Spreadsheet](https://www.uktradeinfo.com/Statistics/OverseasTradeStatistics/AboutOverseastradeStatistics/Documents/IDBR_OTS_tables_2009_to_2012.xls))
1. IDBR OTS Country data (expert users) 2009 - 2012 ([MS Excel Spreadsheet](https://www.uktradeinfo.com/Statistics/OverseasTradeStatistics/AboutOverseastradeStatistics/Documents/IDBR_OTS_Country_2009_to_2012.xls))


'IDBR OTS tables 2015'

dict_keys(['Notes and Contents', 'Industry Group', 'Age Group', 'Employee Size', 'Industry_Age', 'Industry_EmployeeSize', 'EmployeeSize_Age', 'Metadata'])

In [2]:
tab = tabs['Industry_EmployeeSize']

In [3]:
savepreviewhtml(tab)

In [4]:
observations = tab.excel_ref('D10').expand(DOWN).expand(RIGHT).is_not_blank() 

In [5]:
observations = observations - tab.excel_ref('E10').expand(DOWN).is_not_blank()

In [6]:
observations = observations - tab.excel_ref('I10').expand(DOWN).is_not_blank()

In [7]:
observations = observations - tab.excel_ref('F10').expand(DOWN).is_not_blank()

In [8]:
observations = observations - tab.excel_ref('J10').expand(DOWN).is_not_blank()

In [9]:
Industrygroup = tab.excel_ref('A10').expand(DOWN).is_not_blank()

In [10]:
employees = tab.excel_ref('B10').expand(DOWN)

In [11]:
Flow = tab.excel_ref('D6').expand(RIGHT).is_not_blank()

In [12]:
Dimensions = [
            HDimConst('Geography', 'K02000001'),
            HDimConst('Year','2015'),
            HDimConst('Unit', '£ Million'), 
            HDimConst('Measure Type','Total Turnover'),            
            HDim(Industrygroup, 'HMRC Industry', CLOSEST, ABOVE),
            HDim(Flow, 'Flow', CLOSEST, LEFT),
            HDim(employees, 'Employment', DIRECTLY, LEFT)
]

In [13]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)

In [14]:
savepreviewhtml(c1)

In [15]:
new_table = c1.topandas()

In [16]:
new_table.rename(index= str, columns= {'OBS':'Value'}, inplace = True)

In [17]:
new_table['HMRC Industry'] = new_table['HMRC Industry'].map(lambda cell:cell.replace('Grand Total9', 'Total'))

In [18]:
# new_table['HMRC Industry'] = new_table['HMRC Industry'].str.lstrip('Group 1234567890')

In [19]:
# new_table['HMRC Industry'] = new_table['HMRC Industry'].map(lambda cell: cell.replace('and', ''))

In [20]:
new_table = new_table[new_table['Value'] != '']

In [21]:
new_table = new_table[new_table['Value'] != 'S']

In [22]:
new_table['Flow'] = new_table['Flow'].str.rstrip('s')
new_table['Value'] = new_table['Value'].astype(int)

In [23]:
new_table['Employment'] = new_table['Employment'].map(str) + ' employees'

In [24]:
new_table['Employment'] = new_table['Employment'].map(lambda cell: cell.replace('0.0', '0'))

In [25]:
# new_table['HMRC Industry'] = new_table['Industry'].map(lambda x: str(x)[8:])

In [26]:
new_table['HMRC Industry'] = new_table['HMRC Industry'].map(lambda x: str(x)[0:8])

In [27]:
new_table['HMRC Industry'] = new_table['HMRC Industry'].map(lambda cell: cell.replace('Group ', 'group-'))

In [28]:
new_table['HMRC Industry'] = new_table['HMRC Industry'].str.strip()
# new_table['Product'] = new_table['Product'].str.strip()
new_table['Year'] = new_table['Year'].str.strip()
new_table['Flow'] = new_table['Flow'].str.strip()
new_table['Unit'] = new_table['Unit'].str.strip()
new_table['Measure Type'] = new_table['Measure Type'].str.strip()
new_table['Geography'] = new_table['Geography'].str.strip()
new_table['Employment'] = new_table['Employment'].str.strip()

In [29]:
new_table = new_table[['Geography','Year','HMRC Industry','Flow','Employment','Measure Type','Value','Unit']]

In [30]:
# def user_perc(x):
    
#     if x.strip(' ') == '':
#         return 'NA'
#     else:
#         return x
    
# new_table['HMRC Industry'] = new_table.apply(lambda row: user_perc(row['HMRC Industry']), axis = 1)
# new_table['Employment'] = new_table.apply(lambda row: user_perc(row['Employment']), axis = 1)

In [31]:
new_table

Geography  Year HMRC Industry    Flow           Employment  \
0    K02000001  2015       group-1  Export          0 employees   
1    K02000001  2015       group-1  Import          0 employees   
2    K02000001  2015       group-1  Export     1 to 9 employees   
3    K02000001  2015       group-1  Import     1 to 9 employees   
4    K02000001  2015       group-1  Export   10 to 49 employees   
5    K02000001  2015       group-1  Import   10 to 49 employees   
6    K02000001  2015       group-1  Export  50 to 249 employees   
7    K02000001  2015       group-1  Import  50 to 249 employees   
8    K02000001  2015       group-1  Export      250 + employees   
9    K02000001  2015       group-1  Import      250 + employees   
10   K02000001  2015       group-1  Export    Unknown employees   
11   K02000001  2015       group-1  Import    Unknown employees   
13   K02000001  2015       group-2  Import          0 employees   
14   K02000001  2015       group-2  Export     1 to 9 employees   
15   K02000001  2015       group-2  Import     1 to 9 employees   
16   K02000001  2015       group-2  Export   10 to 49 employees   
17   K02000001  2015       group-2  Import   10 to 49 employees   
18   K02000001  2015       group-2  Export  50 to 249 employees   
19   K02000001  2015       group-2  Import  50 to 249 employees   
20   K02000001  2015       group-2  Export      250 + employees   
21   K02000001  2015       group-2  Import      250 + employees   
23   K02000001  2015       group-2  Import    Unknown employees   
24   K02000001  2015       group-3  Export          0 employees   
25   K02000001  2015       group-3  Import          0 employees   
26   K02000001  2015       group-3  Export     1 to 9 employees   
27   K02000001  2015       group-3  Import     1 to 9 employees   
28   K02000001  2015       group-3  Export   10 to 49 employees   
29   K02000001  2015       group-3  Import   10 to 49 employees   
30   K02000001  2015       group-3  Export  50 to 249 employees   
31   K02000001  2015       group-3  Import  50 to 249 employees   
..         ...   ...           ...     ...                  ...   
92   K02000001  2015       group-8  Export      250 + employees   
93   K02000001  2015       group-8  Import      250 + employees   
96   K02000001  2015       group-9  Export          0 employees   
97   K02000001  2015       group-9  Import          0 employees   
98   K02000001  2015       group-9  Export     1 to 9 employees   
99   K02000001  2015       group-9  Import     1 to 9 employees   
100  K02000001  2015       group-9  Export   10 to 49 employees   
101  K02000001  2015       group-9  Import   10 to 49 employees   
102  K02000001  2015       group-9  Export  50 to 249 employees   
103  K02000001  2015       group-9  Import  50 to 249 employees   
104  K02000001  2015       group-9  Export      250 + employees   
105  K02000001  2015       group-9  Import      250 + employees   
106  K02000001  2015       group-9  Export    Unknown employees   
107  K02000001  2015       group-9  Import    Unknown employees   
108  K02000001  2015      group-10  Export          0 employees   
109  K02000001  2015      group-10  Import          0 employees   
110  K02000001  2015      group-10  Export     1 to 9 employees   
111  K02000001  2015      group-10  Import     1 to 9 employees   
112  K02000001  2015      group-10  Export   10 to 49 employees   
113  K02000001  2015      group-10  Import   10 to 49 employees   
114  K02000001  2015      group-10  Export  50 to 249 employees   
115  K02000001  2015      group-10  Import  50 to 249 employees   
116  K02000001  2015      group-10  Export      250 + employees   
117  K02000001  2015      group-10  Import      250 + employees   
118  K02000001  2015      group-10  Export    Unknown employees   
119  K02000001  2015      group-10  Import    Unknown employees   
120  K02000001  2015       Unknown  Export    Unknown employees   
121  K02000001  2015       Unknown  Import    Unknow